In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

In [81]:
df = pd.read_csv("edu_enrollees.csv")
df.head(2)

,enrollee_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target,Xgrp
0,8949.0,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36.0,1.0,train
1,29725.0,city_40,0.776,Male,No relevant experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47.0,0.0,train


### 전처리

단계 1

In [82]:
df = df.drop(columns = ["city", "company_size", "company_type"])
df.head(2)

,enrollee_id,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,last_new_job,training_hours,target,Xgrp
0,8949.0,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,1,36.0,1.0,train
1,29725.0,0.776,Male,No relevant experience,no_enrollment,Graduate,STEM,15,>4,47.0,0.0,train


단계 2

In [83]:
df_na_cnt = pd.DataFrame(dict(types = df.dtypes,
                              cnt = df.isna().sum()))
df_na_cnt

,types,cnt
enrollee_id,float64,0
city_development_index,float64,0
gender,object,4508
relevant_experience,object,0
enrolled_university,object,386
education_level,object,460
major_discipline,object,2813
experience,object,65
last_new_job,object,423
training_hours,float64,0


In [ ]:
df = df.dropna()
df.isna().sum()

Q. .loc로 object인 열만 dropna() 가 가능한가??

In [87]:
df_obj_notna = df.loc[:, df.dtypes == "object"].dropna()
df_join = df_obj_notna.join(df.loc[:, df.dtypes != "object"], 
                            how = "inner")
df_join.head(4)

,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,last_new_job,Xgrp,enrollee_id,city_development_index,training_hours,target
0,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,1,train,8949.0,0.920,36.0,1.0
1,Male,No relevant experience,no_enrollment,Graduate,STEM,15,>4,train,29725.0,0.776,47.0,0.0
4,Male,Has relevant experience,no_enrollment,Masters,STEM,>20,4,train,666.0,0.767,8.0,0.0
7,Male,Has relevant experience,no_enrollment,Graduate,STEM,13,>4,train,402.0,0.762,18.0,1.0


단계 3, 4

In [9]:
df = df.loc[~df["experience"].isin([">20", "<1"]), ]
df["experience"] = df["experience"].astype("int")

df = df.loc[~df["last_new_job"].isin([">4", "never"]), ]
df["last_new_job"] = df["last_new_job"].astype("int")

In [10]:
len(df)

7522

In [11]:
df_base = df.reset_index(drop = True)

### Q1

In [ ]:
df_q1 = df_base[["relevant_experience", "target"]].copy()
df_q1.head(2)

In [13]:
df_q1["relevant_experience"].unique()

array(['Has relevant experience', 'No relevant experience'], dtype=object)

In [14]:
df_A = df_q1.loc[df_q1["relevant_experience"] == "No relevant experience", ]
df_B = df_q1.loc[df_q1["relevant_experience"] == "Has relevant experience", ]
len(df_A), len(df_B)

(1413, 6109)

In [19]:
stat_A = df_A["target"].value_counts(normalize = True)[1]
stat_B = df_B["target"].value_counts(normalize = True)[1]
stat_A, stat_B

(0.38287331917905165, 0.21591095105581928)

In [28]:
sum(df_B["target"] == 1) / len(df_B)

0.21591095105581928

In [21]:
round(stat_A / stat_B, 2)

1.77

In [26]:
df_q1.groupby("relevant_experience")["target"].mean()

relevant_experience
Has relevant experience    0.215911
No relevant experience     0.382873
Name: target, dtype: float64

마크다운(markdown) cell의 commend 모드(파란색)에서 단축키는 "M"  
😉

### Q2.

In [ ]:
# df_base.columns
df_q2_obj = df_base.iloc[:, 2:7]
df_q2_obj.head(2)

In [31]:
# df_q2_dum = pd.get_dummies(df_q2_obj, columns = df_q2_obj.columns) # 시험버전
df_q2_dum = pd.get_dummies(df_q2_obj, columns = df_q2_obj.columns, dtype = "int")
df_q2_dum.head(2)

,gender_Female,gender_Male,gender_Other,relevant_experience_Has relevant experience,relevant_experience_No relevant experience,enrolled_university_Full time course,enrolled_university_Part time course,enrolled_university_no_enrollment,education_level_Graduate,education_level_Masters,education_level_Phd,major_discipline_Arts,major_discipline_Business Degree,major_discipline_Humanities,major_discipline_No Major,major_discipline_Other,major_discipline_STEM
0,0,1,0,1,0,0,0,1,1,0,0,0,0,0,0,0,1
1,0,1,0,1,0,0,0,1,1,0,0,0,0,0,0,0,1


In [ ]:
pd.DataFrame(dict(cols = df_q2_dum.columns))

In [35]:
df_q2_dum = df_q2_dum.drop(columns = df_q2_dum.columns[[2, 4, 7, 10, 16]])
df_q2_dum.head(2)

,gender_Female,gender_Male,relevant_experience_Has relevant experience,enrolled_university_Full time course,enrolled_university_Part time course,education_level_Graduate,education_level_Masters,major_discipline_Arts,major_discipline_Business Degree,major_discipline_Humanities,major_discipline_No Major,major_discipline_Other
0,0,1,1,0,0,1,0,0,0,0,0,0
1,0,1,1,0,0,1,0,0,0,0,0,0


In [36]:
col_t = ["target", "Xgrp"]
col_x = ["city_development_index", "experience", "last_new_job", "training_hours"]
df_job2 = pd.concat([df_base[col_t + col_x], df_q2_dum], axis = 1)

In [37]:
df_job2.head(2)

,target,Xgrp,city_development_index,experience,last_new_job,training_hours,gender_Female,gender_Male,relevant_experience_Has relevant experience,enrolled_university_Full time course,enrolled_university_Part time course,education_level_Graduate,education_level_Masters,major_discipline_Arts,major_discipline_Business Degree,major_discipline_Humanities,major_discipline_No Major,major_discipline_Other
0,1.0,train,0.92,7,1,46.0,0,1,1,0,0,1,0,0,0,0,0,0
1,0.0,train,0.92,5,1,108.0,0,1,1,0,0,1,0,0,0,0,0,0


In [ ]:
model_lr = LogisticRegression(C = 100000, max_iter = 1000, solver = "liblinear", 
                              fit_intercept = True, random_state = 123)
model_lr.fit(X = df_job2.drop(columns = ["target", "Xgrp"]),
             y = df_job2["target"])

In [42]:
np.exp(model_lr.coef_) # OR(Odds Ratio)

array([[0.00220435, 0.97190078, 1.10037739, 0.99907422, 0.84961169,
        0.87221096, 0.46740116, 1.67214935, 0.75465354, 1.38142616,
        1.01688281, 1.29909128, 1.11361023, 1.27567972, 1.48869549,
        0.64443085]])

In [49]:
model_lr.intercept_, model_lr.coef_

(array([4.36024692]),
 array([[-6.11732384e+00, -2.85015530e-02,  9.56531984e-02,
         -9.26206002e-04, -1.62975865e-01, -1.36723955e-01,
         -7.60567385e-01,  5.14109835e-01, -2.81496522e-01,
          3.23116412e-01,  1.67418791e-02,  2.61665001e-01,
          1.07607195e-01,  2.43479152e-01,  3.97900227e-01,
         -4.39387752e-01]]))

In [44]:
np.exp(model_lr.coef_).max()

1.6721493496611732

In [43]:
df_or = pd.DataFrame(np.exp(model_lr.coef_), columns = df_job2.columns[2:])
df_or

,city_development_index,experience,last_new_job,training_hours,gender_Female,gender_Male,relevant_experience_Has relevant experience,enrolled_university_Full time course,enrolled_university_Part time course,education_level_Graduate,education_level_Masters,major_discipline_Arts,major_discipline_Business Degree,major_discipline_Humanities,major_discipline_No Major,major_discipline_Other
0,0.002204,0.971901,1.100377,0.999074,0.849612,0.872211,0.467401,1.672149,0.754654,1.381426,1.016883,1.299091,1.11361,1.27568,1.488695,0.644431


### Q3.

In [46]:
df_train = df_job2.loc[df_job2["Xgrp"] == "train", ]
df_test  = df_job2.loc[df_job2["Xgrp"] == "test", ]
len(df_train), len(df_test)

(4706, 2816)

In [54]:
model_knn = KNeighborsClassifier(n_neighbors = 5, p = 2)
model_knn.fit(X = df_train.drop(columns = ["target", "Xgrp"]),
              y = df_train["target"])
pred = model_knn.predict(df_test.drop(columns = ["target", "Xgrp"]))
pred[:3]

array([0., 0., 0.])

In [55]:
y_t = df_test["target"]
y_p = pred

In [64]:
from sklearn.metrics import accuracy_score

In [67]:
round(accuracy_score(y_true = y_t, y_pred = y_p), 2)
# 0.71

In [76]:
df_tab = pd.crosstab(y_t, y_p)
df_tab

In [79]:
round(df_tab.values.diagonal().sum() / df_tab.values.sum(), 2)

0.71

In [59]:
from sklearn.metrics import pairwise

In [63]:
list(pairwise.distance_metrics().keys()) # knn의 metrics에 사용할 수 있는 명령어(?) 목록

['cityblock',
 'cosine',
 'euclidean',
 'haversine',
 'l2',
 'l1',
 'manhattan',
 'precomputed',
 'nan_euclidean']